In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# convert csv into array
test = pd.read_csv("test.csv", header=None)
test_dict = test.transpose().to_dict()
nested_arr = []
for i in test_dict.keys():
    nested_arr.append([])
for i in test_dict.keys():
    for j in test_dict[i].keys():
        nested_arr[i].append(test_dict[i][j])
nested_arr

[[1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 3, 2, 2, 3, 2, 2, 2],
 [1, 1, 3, 3, 2, 2, 3, 3, 2, 2],
 [9, 1, 3, 3, 3, 3, 3, 3, 4, 2],
 [9, 1, 1, 3, 3, 3, 3, 4, 4, 2],
 [9, 9, 1, 1, 3, 3, 6, 5, 5, 2],
 [9, 8, 8, 10, 3, 3, 6, 6, 5, 2],
 [8, 8, 10, 10, 3, 3, 6, 6, 7, 2],
 [8, 8, 3, 3, 3, 3, 3, 3, 7, 2],
 [8, 7, 7, 7, 7, 7, 7, 7, 7, 7]]

In [3]:
def create_data_model():
    data = {}
    data["constraint_coeffs"] = []
    data["bounds"] = []
    data["obj_coeffs"] = [1 for i in range(len(nested_arr)**2)]
    data["num_vars"] = len(nested_arr)**2
    data["num_constraints"] = 0
    return data

model = create_data_model()

In [4]:
# category constraints
for category in range(len(nested_arr)):
    constraint = []
    for i in range(len(nested_arr)):
        for j in range(len(nested_arr)):
            if nested_arr[i][j] == category + 1:
                constraint.append(1)
            else:
                constraint.append(0)
    # print(category+1, constraint)
    model["constraint_coeffs"].append(constraint)
    model["bounds"].append(1)
    model["num_constraints"] = model["num_constraints"] + 1


In [5]:
# row constraints
for i in range(len(nested_arr)):
    constraint = [0 for i in range(len(nested_arr)**2)]
    for j in range(len(nested_arr)):
        constraint[i*len(nested_arr)+j] = 1
    # print(i+1, constraint)
    model["constraint_coeffs"].append(constraint)
    model["bounds"].append(1)
    model["num_constraints"] = model["num_constraints"] + 1

In [6]:
# column constraints
for i in range(len(nested_arr)):
    constraint = [0 for i in range(len(nested_arr)**2)]
    for j in range(len(nested_arr)):
        constraint[i+len(nested_arr)*j] = 1
    # print(i+1, constraint)
    model["constraint_coeffs"].append(constraint)
    model["bounds"].append(1)
    model["num_constraints"] = model["num_constraints"] + 1

In [7]:
# closeness constraints - right diagonal (for each cell, the immediate diagonal has at most one queen - other adjacencies assured by previous constraints)
for i in range(len(nested_arr)-1): #row
    for j in range(len(nested_arr) - 1): #col
        constraint = [0 for i in range(len(nested_arr)**2)]
        constraint[len(nested_arr)*i+j] = 1
        constraint[len(nested_arr)*(i+1)+j+1] = 1
        # print(i,j,constraint)
        model["constraint_coeffs"].append(constraint)
        model["bounds"].append(1)
        model["num_constraints"] = model["num_constraints"] + 1

In [8]:
# closeness constraints - left diagonal
for i in range(len(nested_arr)-1):
    for j in range(1,len(nested_arr)):
        constraint = [0 for i in range(len(nested_arr)**2)]
        constraint[len(nested_arr)*i+j] = 1
        constraint[len(nested_arr)*(i+1)+j-1] = 1
        # print(i,j,constraint)
        model["constraint_coeffs"].append(constraint)
        model["bounds"].append(1)
        model["num_constraints"] = model["num_constraints"] + 1

In [9]:
solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("no")

In [10]:
# initialize variables
x = {}
for j in range(model["num_vars"]):
    x[j] = solver.IntVar(0, 1, "x[%i]" % j)
print("Number of variables =", solver.NumVariables())
print(x)

Number of variables = 100
{0: x[0], 1: x[1], 2: x[2], 3: x[3], 4: x[4], 5: x[5], 6: x[6], 7: x[7], 8: x[8], 9: x[9], 10: x[10], 11: x[11], 12: x[12], 13: x[13], 14: x[14], 15: x[15], 16: x[16], 17: x[17], 18: x[18], 19: x[19], 20: x[20], 21: x[21], 22: x[22], 23: x[23], 24: x[24], 25: x[25], 26: x[26], 27: x[27], 28: x[28], 29: x[29], 30: x[30], 31: x[31], 32: x[32], 33: x[33], 34: x[34], 35: x[35], 36: x[36], 37: x[37], 38: x[38], 39: x[39], 40: x[40], 41: x[41], 42: x[42], 43: x[43], 44: x[44], 45: x[45], 46: x[46], 47: x[47], 48: x[48], 49: x[49], 50: x[50], 51: x[51], 52: x[52], 53: x[53], 54: x[54], 55: x[55], 56: x[56], 57: x[57], 58: x[58], 59: x[59], 60: x[60], 61: x[61], 62: x[62], 63: x[63], 64: x[64], 65: x[65], 66: x[66], 67: x[67], 68: x[68], 69: x[69], 70: x[70], 71: x[71], 72: x[72], 73: x[73], 74: x[74], 75: x[75], 76: x[76], 77: x[77], 78: x[78], 79: x[79], 80: x[80], 81: x[81], 82: x[82], 83: x[83], 84: x[84], 85: x[85], 86: x[86], 87: x[87], 88: x[88], 89: x[89], 90:

In [11]:
# initialize constraints
for i in range(model["num_constraints"]):
    constraint = solver.RowConstraint(0, model["bounds"][i], "")
    for j in range(model["num_vars"]):
        constraint.SetCoefficient(x[j], model["constraint_coeffs"][i][j])
print("Number of constraints =", solver.NumConstraints())

Number of constraints = 192


In [12]:
# set objective
objective = solver.Objective()
for j in range(model["num_vars"]):
    objective.SetCoefficient(x[j], model["obj_coeffs"][j])
objective.SetMaximization()

In [13]:
# solve
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with SCIP 9.0.0 [LP solver: Glop 9.10]


In [14]:
if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())
    for j in range(model["num_vars"]):
        print(x[j].name(), " = ", x[j].solution_value())
    print()
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    print(f"Problem solved in {solver.nodes():d} branch-and-bound nodes")
else:
    print("The problem does not have an optimal solution.")

Objective value = 10.0
x[0]  =  0.0
x[1]  =  0.0
x[2]  =  0.0
x[3]  =  0.0
x[4]  =  1.0
x[5]  =  0.0
x[6]  =  0.0
x[7]  =  0.0
x[8]  =  0.0
x[9]  =  0.0
x[10]  =  0.0
x[11]  =  0.0
x[12]  =  0.0
x[13]  =  0.0
x[14]  =  0.0
x[15]  =  0.0
x[16]  =  0.0
x[17]  =  0.0
x[18]  =  0.0
x[19]  =  1.0
x[20]  =  0.0
x[21]  =  0.0
x[22]  =  1.0
x[23]  =  0.0
x[24]  =  0.0
x[25]  =  0.0
x[26]  =  0.0
x[27]  =  0.0
x[28]  =  0.0
x[29]  =  0.0
x[30]  =  0.0
x[31]  =  0.0
x[32]  =  0.0
x[33]  =  0.0
x[34]  =  0.0
x[35]  =  0.0
x[36]  =  0.0
x[37]  =  0.0
x[38]  =  1.0
x[39]  =  0.0
x[40]  =  1.0
x[41]  =  0.0
x[42]  =  0.0
x[43]  =  0.0
x[44]  =  0.0
x[45]  =  0.0
x[46]  =  0.0
x[47]  =  0.0
x[48]  =  0.0
x[49]  =  0.0
x[50]  =  0.0
x[51]  =  0.0
x[52]  =  0.0
x[53]  =  0.0
x[54]  =  0.0
x[55]  =  0.0
x[56]  =  0.0
x[57]  =  1.0
x[58]  =  0.0
x[59]  =  0.0
x[60]  =  0.0
x[61]  =  0.0
x[62]  =  0.0
x[63]  =  1.0
x[64]  =  0.0
x[65]  =  0.0
x[66]  =  0.0
x[67]  =  0.0
x[68]  =  0.0
x[69]  =  0.0
x[70]  

In [15]:
# Print solution neatly
sol_arr = []
row = []
for j in range(model["num_vars"]):     
        row.append(1 if x[j].solution_value() == 1.0 else 0)
        if len(row) == len(nested_arr):
                sol_arr.append(row)
                row = []
sol_arr

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]